In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
# include("../../../functions/func_map.jl")
# include("../../../functions/func_stat.jl")
# include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure3/distribution_summary"
mkpath(fig_path)

In [ ]:
review_fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/review_figures"
mkpath(review_fig_path)

In [ ]:
datasets_corner_cue = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];

In [ ]:
data_info_all = reduce(vcat, [datasets_corner_cue])

In [ ]:
analyzer = "chuyu"

In [ ]:
function neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
    
    # for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
    region_roi_bool = region_roi_bool[:,findall(region_names .== region_name)][:,1]
    whether_region = falses(n_neuron)
    for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
        if sum(region_roi_bool[neuron_label.==which_neuron]) >0
            whether_region[which_neuron] = true
        end
    end
    return whether_region
end

In [ ]:
which_data = 1
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]
experimenter = data_info[end]

ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
close(NMF_file)

n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)



all_files = readdir(data_path(ds_save_cy_1))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
close(file)
    

    
place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01));

In [ ]:
plot_loc(place_cell_index_1;valid_neurons = valid_roi_1)

In [ ]:
println(region_names)

In [ ]:
whether_region_all = []
for region_name in which_regions
    whether_region = neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
    append!(whether_region_all, [whether_region])
end

In [ ]:
for whether_region in whether_region_all
    plot_loc(findall(whether_region);valid_neurons = valid_roi_1)
end

In [ ]:
place_cell_percentage_all = []
for whether_region in whether_region_all
    region_neurons = findall(whether_region)
    place_cell_percentage = sum(whether_in(region_neurons, place_cell_index_1))./length(region_neurons)
    append!(place_cell_percentage_all, place_cell_percentage)
end

In [ ]:
place_cell_percentage_all

In [ ]:
region_place_cell_nr = [length(intersect(place_cell_index_1, findall(whether_region_all[i]))) for i in 1:length(whether_region_all)]
fraction_place_cell = region_place_cell_nr./length(place_cell_index_1)

# For all fish, large region


In [ ]:
which_regions = ["Telencephalon -", "Diencephalon -", "Mesencephalon -", "Rhombencephalon -"]

In [ ]:
place_cell_percentage_all_fish = []
fraction_tel_place_cell_all_fish = []
for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_neuron = length(X_all);

    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)



    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)



    place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01));



    whether_region_all = []
    for region_name in which_regions
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
        append!(whether_region_all, [whether_region])
    end

    place_cell_percentage_all = []
    for whether_region in whether_region_all
        region_neurons = findall(whether_region)
        place_cell_percentage = sum(whether_in(region_neurons, place_cell_index_1))./length(region_neurons)
        append!(place_cell_percentage_all, place_cell_percentage)
    end
    
    
    region_place_cell_nr = [length(intersect(place_cell_index_1, findall(whether_region_all[i]))) for i in 1:length(whether_region_all)]
    fraction_place_cell = region_place_cell_nr./length(place_cell_index_1)
    
    
    append!(place_cell_percentage_all_fish , [place_cell_percentage_all])
    append!(fraction_tel_place_cell_all_fish , [fraction_place_cell])
    
    
    # h5open(joinpath(data_path(ds_save_analyzer_1), "place_cell_percentage.h5"), "w") do file
    #     file["place_cell_percentage_all"] = Float32.(place_cell_percentage_all)
    #     file["fraction_place_cell"] = Float32.(fraction_place_cell)
    #     file["which_regions"] = which_regions

    # end;
    
end

In [ ]:
data_info_all

In [ ]:
place_cell_percentage_all_fish

In [ ]:
fraction_tel_place_cell_all_fish

In [ ]:
mean([fraction_tel_place_cell_all_fish[i][1] for i in 1:length(fraction_tel_place_cell_all_fish)])

In [ ]:
std([fraction_tel_place_cell_all_fish[i][1] for i in 1:length(fraction_tel_place_cell_all_fish)])

In [ ]:
interested_large_regions = ["Tel.", "Di.", "Mes.", "Rhomb."]
region_percentage_all = []
for i in 1:length(place_cell_percentage_all_fish)
    
    region_percentage = Dict()
    for (i_region, region_name) in enumerate(interested_large_regions)
        region_percentage[region_name] = place_cell_percentage_all_fish[i][i_region]
    end
    
    append!(region_percentage_all,[region_percentage])
end

region_fraction_all = []
for i in 1:length(fraction_tel_place_cell_all_fish)
    region_fraction = Dict()
    for (i_region, region_name) in enumerate(interested_large_regions)
        region_fraction[region_name] = fraction_tel_place_cell_all_fish[i][i_region]
    end
    
    append!(region_fraction_all,[region_fraction])
end

In [ ]:
region_fraction_all_large = copy(region_fraction_all)

In [ ]:
region_fraction_all_large = copy(region_fraction_all)

In [ ]:
@pyimport pandas as pd
@pyimport seaborn as sns

In [ ]:
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
hot = cm.get_cmap("hot", 100)
alpha_top = 50
cmaplist = [hot(i) for i in 1:(hot.N)]
cmaplist_new = []
length_cmaplist = length(cmaplist)
for (i, color_) in enumerate(cmaplist)
    new_color_ = numpy.copy(color_)
    if i<=alpha_top
        new_color_[4] = (100/alpha_top)*i/length_cmaplist
    end
    append!(cmaplist_new,[new_color_]) 
end
hot_new = mpl_colors.LinearSegmentedColormap.from_list("hot_new",cmaplist_new,100)

In [ ]:
region_percentage_all

In [ ]:
df = pd.DataFrame(region_percentage_all)
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")
fig, ax = subplots(1,1,figsize=(1,2))
sns.stripplot(order = interested_large_regions, data=df,ax=ax, color=hot_new(60), s=3, alpha=0.8)

ax.set_ylabel("# PCs per region / # cells per region")
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)



order = interested_large_regions
for i in 0:length(order)-1
    m = numpy.nanmedian([x[order[i+1]] for x in region_percentage_all])
    ax.plot([i-0.3, i+0.3], [m, m], c="k", lw=2)
end

xticks([0,1,2,3], labels= ["Tel.", "Di.", "Mes.", "Rhomb."], rotation=45, ha="right" , rotation_mode="anchor")
# yticks([0,500,1000,1500,2000],[0,"","","",2000])

fig.savefig(joinpath(fig_path, "region_percentage.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
df = pd.DataFrame(region_fraction_all)
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")
fig, ax = subplots(1,1,figsize=(1,2))
sns.stripplot(order = interested_large_regions, data=df,ax=ax, color=hot_new(60), s=3, alpha=0.8)

ax.set_ylabel("# PCs per region / # PCs")
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)



order = interested_large_regions
for i in 0:length(order)-1
    m = numpy.nanmedian([x[order[i+1]] for x in region_fraction_all])
    ax.plot([i-0.3, i+0.3], [m, m], c="k", lw=2)
end

xticks([0,1,2,3], labels= ["Tel.", "Di.", "Mes.", "Rhomb."], rotation=45, ha="right" , rotation_mode="anchor")

ax.set_ylim(-0.02, 1.02)
# yticks([0,500,1000,1500,2000],[0,"","","",2000])

fig.savefig(joinpath(review_fig_path, "region_fraction.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

# For all fish, tel region


In [ ]:
which_regions = ["Telencephalon - Pallium", "Telencephalon - Subpallium"]

In [ ]:
place_cell_percentage_all_fish = []
fraction_tel_place_cell_all_fish = []
for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_neuron = length(X_all);

    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)



    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)



    place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01));



    whether_region_all = []
    for region_name in which_regions
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
        append!(whether_region_all, [whether_region])
    end

    place_cell_percentage_all = []
    for whether_region in whether_region_all
        region_neurons = findall(whether_region)
        place_cell_percentage = sum(whether_in(region_neurons, place_cell_index_1))./length(region_neurons)
        append!(place_cell_percentage_all, place_cell_percentage)
    end
    
    
    region_place_cell_nr = [length(intersect(place_cell_index_1, findall(whether_region_all[i]))) for i in 1:length(whether_region_all)]
    fraction_place_cell = region_place_cell_nr./length(place_cell_index_1)
    
    
    append!(place_cell_percentage_all_fish , [place_cell_percentage_all])
    append!(fraction_tel_place_cell_all_fish , [fraction_place_cell])
end

In [ ]:
place_cell_percentage_all_fish

In [ ]:
fraction_tel_place_cell_all_fish

In [ ]:
interested_large_regions = ["Pallium", "Subpallium"]
region_percentage_all = []
for i in 1:length(place_cell_percentage_all_fish)
    
    region_percentage = Dict()
    for (i_region, region_name) in enumerate(interested_large_regions)
        region_percentage[region_name] = place_cell_percentage_all_fish[i][i_region]
    end
    
    append!(region_percentage_all,[region_percentage])
end

region_fraction_all = []
for i in 1:length(fraction_tel_place_cell_all_fish)
    region_fraction = Dict()
    for (i_region, region_name) in enumerate(interested_large_regions)
        region_fraction[region_name] = fraction_tel_place_cell_all_fish[i][i_region]
    end
    
    append!(region_fraction_all,[region_fraction])
end

In [ ]:
region_fraction_all_small = copy(region_fraction_all)

In [ ]:
@pyimport pandas as pd
@pyimport seaborn as sns

In [ ]:
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
hot = cm.get_cmap("hot", 100)
alpha_top = 50
cmaplist = [hot(i) for i in 1:(hot.N)]
cmaplist_new = []
length_cmaplist = length(cmaplist)
for (i, color_) in enumerate(cmaplist)
    new_color_ = numpy.copy(color_)
    if i<=alpha_top
        new_color_[4] = (100/alpha_top)*i/length_cmaplist
    end
    append!(cmaplist_new,[new_color_]) 
end
hot_new = mpl_colors.LinearSegmentedColormap.from_list("hot_new",cmaplist_new,100)

In [ ]:
interested_large_regions

In [ ]:
df = pd.DataFrame(region_percentage_all)
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")
fig, ax = subplots(1,1,figsize=(0.75,2))
sns.stripplot(order = interested_large_regions, data=df,ax=ax, color=hot_new(60), s=3, alpha=0.8)

ax.set_ylabel("# PCs per region / # cells per region")
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)



order = interested_large_regions
for i in 0:length(order)-1
    m = numpy.nanmedian([x[order[i+1]] for x in region_percentage_all])
    ax.plot([i-0.3, i+0.3], [m, m], c="k", lw=2)
end

xticks([0,1], labels= ["Pallium", "Dorsal subpallium"], rotation=45, ha="right" , rotation_mode="anchor")
# yticks([0,500,1000,1500,2000],[0,"","","",2000])
ylim(-0.01, 0.4)
fig.savefig(joinpath(fig_path, "region_percentage_pallium_subplallium.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
df = pd.DataFrame(region_fraction_all)
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")
fig, ax = subplots(1,1,figsize=(0.5,2))
sns.stripplot(order = interested_large_regions, data=df,ax=ax, color=hot_new(60), s=3, alpha=0.8)

# ax.set_ylabel("# PCs per region / # PCs")
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)



order = interested_large_regions
for i in 0:length(order)-1
    m = numpy.nanmedian([x[order[i+1]] for x in region_fraction_all])
    ax.plot([i-0.3, i+0.3], [m, m], c="k", lw=2)
end

xticks([0,1], labels= ["Pallium", "Dorsal subpallium"], rotation=45, ha="right" , rotation_mode="anchor")
# yticks([0,500,1000,1500,2000],[0,"","","",2000])
ax.set_ylim(-0.02, 1.02)
yticks([])


fig.savefig(joinpath(review_fig_path, "region_fraction_pallium_subplallium.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
data_tog = []
for i = 1:length(region_fraction_all_large)
    append!(data_tog, [Dict(region_fraction_all_small[i]..., region_fraction_all_large[i]...)])
end



In [ ]:
df =  pd.DataFrame.from_dict(data_tog)

In [ ]:
# desired_order = ["Tel.", "Di.", "Mes.", "Rhomb.", "Pallium", "Subpallium"]

In [ ]:
@pyimport os
@pyimport openpyxl

save_path = "../../../figures/source_data/Source Data Fig. 1.xlsx"
sheet_name = "Fig. 1g"

exists = os.path.isfile(save_path)
if exists
    writer = pd.ExcelWriter(save_path, engine = "openpyxl", mode="a", if_sheet_exists="replace")
    df.to_excel(writer, sheet_name = sheet_name)
    writer.close()
else
    df.to_excel(save_path, sheet_name)  
end